In [1]:
!pip install streamlit
!pip install img2vec_pytorch
!pip install scikit-learn==1.4.2
!pip install st_pages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.

In [11]:
%%writefile croprecom.py
import numpy as np
import pickle
import streamlit as st
import sklearn
model=pickle.load(open("trained_model.sav",'rb'))
#model=pickle.load(open("..train_model.sav",'rb'))

crops=['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee']
crops.sort()
labels=[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21]
label_crops=dict(zip(labels,crops))
html_code = '''
<h1 style="color:blue; text-align:center">Crop Recommendation System</h1>
'''
## Function to predict which crop is best suited for particular region
def CropRecommendation(input_data):

    input_data=np.array(input_data).reshape(1,-1)
    recommend=model.predict(input_data)
    print(recommend)
    print(label_crops[recommend[0]])
    return label_crops[recommend[0]]




def main():
    st.markdown(html_code,unsafe_allow_html=True)



    #Required Data
# Nitrogen, Phosphorous,Potassium,Temperature,Rainfall,Ph
    nitrogen=st.text_input("Enter Nitrogen content in soil ")
    phosphorous=st.text_input("Enter Phosphorous content in soil ")
    potassium=st.text_input("Enter Potassium content in soil ")
    temperature=st.text_input("Enter Temperature in Celsius")
    humidity=st.text_input("Enter relative humidity in %")
    ph=st.text_input("Enter ph value of the soil")
    rainfall=st.text_input("Enter rainfall in mm")

    BestCrop=""
    if st.button("Recommend Crop"):

        print(nitrogen)
        if( nitrogen and  phosphorous and  potassium and  temperature and  rainfall and ph and humidity):
            BestCrop=CropRecommendation([int(nitrogen),int(phosphorous),int(potassium),float(temperature),float(humidity),float(ph),float(rainfall)])
            st.success(BestCrop)
        else :
            st.write("Enter Correct Values")
if __name__=='__main__':
    main()

Overwriting croprecom.py


In [12]:
%%writefile analyzer.py

import streamlit as st
import pickle
import numpy as np
from nltk.classify import NaiveBayesClassifier

# Define features (words) and their corresponding labels (emotions)
def word_features(words):
    return dict([(word, True) for word in words])

# Define emotions and their associated words
emotions = {
    'happy': ['happy', 'joyful', 'excited'],
    'sad': ['sad', 'unhappy', 'depressed'],
    'angry': ['angry', 'mad', 'furious'],
    'excited': ['excited', 'thrilled', 'eager'],
    'nervous': ['nervous', 'anxious', 'worried'],
    'scared': ['scared', 'fearful', 'terrified']
}

# Generate training set for each emotion
train_set = [(word_features(word.split()), emotion)
             for emotion, words in emotions.items()
             for word in words]

# Train the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(train_set)

# Function to predict emotions
def predict_emotions(docx):
    results = classifier.classify(word_features(docx.split()))
    return results

# Function to get prediction probabilities
def get_prediction_proba(docx):
    results = classifier.prob_classify(word_features(docx.split()))
    probabilities = {label: results.prob(label) for label in results.samples()}
    return probabilities

# Function to save the trained classifier to a pickle file
def save_model_to_pickle(model, filename):
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

# Main Application
def main():
    st.title("Emotion Classifier App")

    # Text input for user to enter a sentence
    sentence = st.text_input("Enter a sentence:")

    if sentence:
        # Classify the sentence
        sentiment = predict_emotions(sentence)
        probabilities = get_prediction_proba(sentence)

        st.write(f"Predicted Emotion: {sentiment}")

        st.write("Prediction Probabilities:")
        for emotion, probability in probabilities.items():
            st.write(f"{emotion}: {probability:.4f}")

        # Button to save the trained classifier to a pickle file
        if st.button("Save Model"):
            save_model_to_pickle(classifier, 'trained_classifier.pkl')
            st.success("Model saved to 'trained_classifier.pkl'")

if __name__ == '__main__':
    main()


Overwriting analyzer.py


In [13]:
%%writefile classify.py

from PIL import Image
import streamlit as st
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

st.title("Fruit Identification")
st.header("Supported fruits: lemon, apple, mandarin, orange")
st.text("Upload a clear image of a fruit")

# Load fruit data
fruit_data = pd.read_csv("fruit.csv", header=None, names=["fruit_label", "mass", "width", "height", "color_score", "fruit_name"])

# Load the model if it exists
if os.path.exists('fruit_classifier.sav'):
    model = joblib.load('fruit_classifier.sav')
else:
    model = None

uploaded_file = st.file_uploader("Enter image", type=["png", "jpeg", "jpg"])

def simulate_features(image):
    """Simulate mass and color score features for demonstration purposes."""
    # Simulating feature extraction process
    mass = np.random.randint(50, 500)  # Random mass between 50 and 500 grams
    color_score = np.random.uniform(0, 1)  # Random color score between 0 and 1

    return mass, color_score

def classify_fruit(image):
    """Classify the fruit based on the simulated features."""
    if model is None:
        return "Model not trained yet."

    # Simulate feature extraction from the image
    mass, color_score = simulate_features(image)

    # Prepare feature vector
    features = np.array([[mass, color_score]])

    # Predict using the trained model
    fruit_label = model.predict(features)[0]
    fruit_name = fruit_data[fruit_data['fruit_label'] == fruit_label]['fruit_name'].values[0]

    return fruit_name

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded image', use_column_width=True)
    st.write("")
    st.write("Classifying...")

    predicted_fruit = classify_fruit(image)

    if predicted_fruit:
        st.write(f"The uploaded fruit is {predicted_fruit}.")
    else:
        st.write("Fruit classification failed.")

    # Prepare training data
    X = fruit_data[['mass', 'color_score']]
    y = fruit_data['fruit_label']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    st.write(f"Model accuracy: {accuracy}")

    # Save the trained model as a .sav file
    joblib.dump(model, 'fruit_classifier.sav')
    st.write("Model saved as 'fruit_classifier.sav'")


Overwriting classify.py


In [14]:
%%writefile croprecom_src.py
import streamlit as st

st.header('Crop Recommendation App')
st.subheader('This model was trained using a dataset')
st.code('''
import numpy as np
import pickle
import streamlit as st
import sklearn
model = pickle.load(open("trained_model.sav", 'rb'))
# model = pickle.load(open("..train_model.sav", 'rb'))

crops = ['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
         'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
         'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
         'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee']
crops.sort()
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
          17, 18, 19, 20, 21]
label_crops = dict(zip(labels, crops))

## Function to predict which crop is best suited for particular region
def CropRecommendation(input_data):
    input_data = np.array(input_data).reshape(1, -1)
    recommend = model.predict(input_data)
    print(recommend)
    print(label_crops[recommend[0]])
    return label_crops[recommend[0]]

def main():
    st.markdown(html_code, unsafe_allow_html=True)

    # Required Data
    # Nitrogen, Phosphorous, Potassium, Temperature, Rainfall, Ph
    nitrogen = st.text_input("Enter Nitrogen content in soil ")
    phosphorous = st.text_input("Enter Phosphorous content in soil ")
    potassium = st.text_input("Enter Potassium content in soil ")
    temperature = st.text_input("Enter Temperature in Celsius")
    humidity = st.text_input("Enter relative humidity in %")
    ph = st.text_input("Enter ph value of the soil")
    rainfall = st.text_input("Enter rainfall in mm")

    BestCrop = ""
    if st.button("Recommend Crop"):
        if (nitrogen and phosphorous and potassium and temperature and rainfall and ph and humidity):
            BestCrop = CropRecommendation([int(nitrogen), int(phosphorous), int(potassium), float(temperature), float(humidity), float(ph), float(rainfall)])
            st.success(BestCrop)
        else:
            st.write("Enter Correct Values")

if __name__ == '__main__':
    main()
''')


Overwriting croprecom_src.py


In [15]:
%%writefile analyzer_src.py
import streamlit as st


st.header('Sentiment Analyzer App')
st.subheader('This model was trained using a dataset')
st.code('''

import streamlit as st
import pickle
import numpy as np
from nltk.classify import NaiveBayesClassifier

# Define features (words) and their corresponding labels (emotions)
def word_features(words):
    return dict([(word, True) for word in words])

# Define emotions and their associated words
emotions = {
    'happy': ['happy', 'joyful', 'excited'],
    'sad': ['sad', 'unhappy', 'depressed'],
    'angry': ['angry', 'mad', 'furious'],
    'excited': ['excited', 'thrilled', 'eager'],
    'nervous': ['nervous', 'anxious', 'worried'],
    'scared': ['scared', 'fearful', 'terrified']
}

# Generate training set for each emotion
train_set = [(word_features(word.split()), emotion)
             for emotion, words in emotions.items()
             for word in words]

# Train the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(train_set)

# Function to predict emotions
def predict_emotions(docx):
    results = classifier.classify(word_features(docx.split()))
    return results

# Function to get prediction probabilities
def get_prediction_proba(docx):
    results = classifier.prob_classify(word_features(docx.split()))
    probabilities = {label: results.prob(label) for label in results.samples()}
    return probabilities

# Function to save the trained classifier to a pickle file
def save_model_to_pickle(model, filename):
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

# Main Application
def main():
    st.title("Emotion Classifier App")

    # Text input for user to enter a sentence
    sentence = st.text_input("Enter a sentence:")

    if sentence:
        # Classify the sentence
        sentiment = predict_emotions(sentence)
        probabilities = get_prediction_proba(sentence)

        st.write(f"Predicted Emotion: {sentiment}")

        st.write("Prediction Probabilities:")
        for emotion, probability in probabilities.items():
            st.write(f"{emotion}: {probability:.4f}")

        # Button to save the trained classifier to a pickle file
        if st.button("Save Model"):
            save_model_to_pickle(classifier, 'trained_classifier.pkl')
            st.success("Model saved to 'trained_classifier.pkl'")

if __name__ == '__main__':
    main()
''')

Overwriting analyzer_src.py


In [16]:
%%writefile classify_src.py
import streamlit as st


st.header('Image Classification App')
st.subheader('This model was trained using a dataset')
st.code('''

# overwrite app.py

from PIL import Image
import streamlit as st
import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

st.title("Fruit Identification")
st.header("Supported fruits: lemon, apple, mandarin, orange")
st.text("Upload a clear image of a fruit")

fruit_data = pd.read_csv("fruit.csv", header=None, names=["fruit_label", "mass", "width", "height", "color_score", "fruit_name"])

uploaded_file = st.file_uploader("Enter image", type=["png", "jpeg", "jpg"])

def classify_fruit(image):
    # Function to classify the fruit based on the fruit data

    # Assuming you have a method to extract features from the image
    # Here we'll just use random values for demonstration
    mass = np.random.randint(50, 500)  # Random mass between 50 and 500 grams
    color_score = np.random.uniform(0, 1)  # Random color score between 0 and 1

    # Calculate the Euclidean distance between the features of the image and each fruit in the dataset
    fruit_data['distance'] = np.sqrt((fruit_data['mass'] - mass)**2 +
                                     (fruit_data['color_score'] - color_score)**2)

    # Find the closest fruit based on the calculated distance
    closest_fruit_index = fruit_data['distance'].idxmin()
    closest_fruit = fruit_data.loc[closest_fruit_index, 'fruit_name']

    return closest_fruit

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded image', use_column_width=True)
    st.write("")
    st.write("Classifying...")

    predicted_fruit = classify_fruit(image)

    if predicted_fruit:
        st.write(f"The uploaded fruit is {predicted_fruit}.")
    else:
        st.write("Fruit classification failed.")

    # Create features (for demonstration purpose)
    X = fruit_data[['mass', 'color_score']]
    y = fruit_data['fruit_label']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    st.write(f"Model accuracy: {accuracy}")

    # Save the trained model as a .sav file
    joblib.dump(model, 'fruit_classifier.sav')

    st.write("Model saved as 'fruit_classifier.sav'")
''')

Overwriting classify_src.py


In [32]:
%%writefile home.py
import streamlit as st
from st_pages import Page, Section, show_pages, add_page_title, hide_pages

add_page_title()
show_pages(
    [
        Page("home.py", "ITEQMT Machine Learning Application Portfolio", "💻"),
        Section("Machine Learning UI App", "🧙‍♂️"),
        Page("croprecom.py", "Crop Recommendation ML Model", "1️⃣", in_section=True),
        Page("analyzer.py", "Basic Sentiment Analyzer", "2️⃣", in_section=True),
        Page("classify.py", "Image Classification", "3️⃣", in_section=True),

        Section("Sample Source Code", "💾"),
        Page("croprecom_src.py", "Crop Recommendation SRC", "1️⃣", in_section=True),
        Page("analyzer_src.py", "Basic Sentiment Analyzer SRC", "2️⃣", in_section=True),
        Page("classify_src.py", "Image Classification SRC", "3️⃣", in_section=True),
    ]
)

hide_pages(["Thank you"])

st.markdown("### 👨‍🔧 ML Learning by [JESSATOQUERO](https://github.com/JESSATOQUERO)")

st.image("./back.jpg")
st.info("👨‍🔧 Please take note when on streamlit.app the [Image Classification] pages are not working due to Memory Limitation of 'Free Tier' hosting of Streamlit")
st.markdown("---")

with st.expander("About""Myself"""):
    st.markdown("""

    #

      ### About Myself
        Name: Jessa O. Toquero
        Age: 23
        School: Carlos Hilado Memorial State University
        Course, Year&Section: BSIS 3B
        Skills Learned: Html&css, js, java, php, sql, c++, and python

   # Contact information
st.info("For more info, contact [Toquero, Jessa O.](https://www.facebook.com/jessa.toquero.942?mibextid=JRoKGi) on Facebook", icon="📧")
st.info("Contact [Toquero, Jessa O.](https://www.instagram.com/toquero.jessa?igsh=MXdnaGY0Z2hzNzJqag==) on Instagram", icon="📧")
st.warning("Please note that when on Streamlit.app, the [Image Classification] pages might not work due to the memory limitations of 'Free Tier' hosting on Streamlit.")
st.markdown("---")
    #""", unsafe_allow_html=True)


st.image("./ml.jpg")
st.markdown("""
### 👨‍🎓 Read More


##### 👨‍👦‍👦 Description of Apps

* Prediction App is about the crop recommendation machine learning based on the required input in croprecom.py file
* Sentiment Analyzer is about the emotion identifier of information input of how you feel or what have will you do, etc. in analyzer.py file
* Image Classification is about the determination wether the uploaded picture is an apple, orange, lemon, or mandarin.

##### 👨‍🔧 What I have Learned

 Machine learning has a rich history spanning several decades. It began in the 1940s-1950s with the foundation of AI and the introduction of neural networks by Frank Rosenblatt.
        The 1950s-1960s saw symbolic AI using explicit rules and logic, with early programs like the Logic Theorist. From the 1960s-1980s, machine learning algorithms emerged, including the nearest neighbor algorithm and the backpropagation algorithm for neural networks. Despite an "AI Winter" in the 1970s-1980s, foundational work continued. The 1990s brought a resurgence with statistical methods like SVMs and ensemble methods.
        The 2000s-2010s marked the deep learning revolution with CNNs and RNNs, significantly advancing image and sequence data processing. By the 2010s, machine learning became crucial in various fields, aided by frameworks like TensorFlow and PyTorch.
        Key figures include Geoffrey Hinton, Yann LeCun, and Yoshua Bengio, pioneers in deep learning. Today, machine learning continues to evolve rapidly, driving innovation across numerous applications.

        ### Purpose of Machine Learning

        The purpose of machine learning is to enable computers to learn from data and make decisions or predictions without being explicitly programmed.
        It aims to create systems that can automatically improve their performance over time through experience. Machine learning is used to recognize patterns, classify information, and make forecasts based on data. It helps in automating tasks, enhancing efficiency, and providing insights in various fields such as healthcare, finance, marketing, and technology. By leveraging algorithms and statistical models, machine learning enables the development of intelligent applications that can adapt and respond to new data, ultimately driving innovation and solving complex problems.

        ### Usage of Machine Learning
        Machine learning is used across various industries and applications, including:

        * Healthcare: Machine learning is used to analyze medical data for diagnosing diseases, predicting patient outcomes, and personalizing treatment plans.
        * Entertainment: It powers recommendation systems for streaming services, helping to suggest movies, shows, and music tailored to individual preferences.
        * Transportation: Machine learning enhances autonomous driving technologies, optimizing routes, and improving safety through predictive maintenance and traffic management.
        * Retail: Machine learning helps in demand forecasting, personalized marketing, and inventory management, enhancing customer experience and sales.
        * Manufacturing: It enables predictive maintenance, quality control, and supply chain optimization, improving efficiency and reducing downtime.
        * Business: It is used for financial analysis, fraud detection, and customer relationship management, driving better decision-making and operational efficiency.
        """





)

### 🔎 Overview""", unsafe_allow_html=True)

st.image("./back.jpg")

st.markdown("""
              My name is Jessa O. Toquero, a 23-year-old student from Carlos Hilado Memorial State University,
              currently enrolled in BSIS 3B. I have acquired various skills in programming languages such as HTML & CSS,
              JavaScript, Java, PHP, SQL, C++, and Python. Through the development of my applications, I have created a
              Prediction app that uses trained models for forecasting and estimating outcomes, a Sentiment Analyzer that
              determines the sentiment of text data, and an Image Classification app that categorizes images using machine
              learning models. These projects have deepened my understanding of algorithmic processes, data preparation,
              model evaluation, and the deployment of machine learning models in user-friendly interfaces. Additionally, I
              have improved my skills in building interactive applications with Streamlit, furthering my knowledge in both
              programming and machine learning domains.
""", unsafe_allow_html=True)

hide_streamlit_style = """
<style>
#MainMenu {visibility: hidden;}
footer {visibility: hidden;}
</style>
"""

# st.markdown(hide_streamlit_style, unsafe_allow_html=True)


Overwriting home.py


In [33]:
! wget -q -O - ipv4.icanhazip.com

34.83.45.192


In [34]:
! streamlit run home.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.45.192:8501

npx: installed 22 in 3.352s
your url is: https://warm-schools-sing.loca.lt
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GaussianNB from version 1.2.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
  Stopping...
^C


In [39]:
!pip install streamlit


In [40]:
%%writefile app.py
import streamlit as st

def main():
    st.title("Hello Streamlit!")
    st.write("Welcome to Streamlit.")

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!streamlit run --server.port 8502 app.py






  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.83.45.192:8502

